In [79]:
import pymysql
import pandas as pd
import re
from bs4 import BeautifulSoup
from hanziconv import HanziConv
from zhon import hanzi
import string
punctuation=hanzi.punctuation + string.punctuation
punctuation = set([i for i in punctuation])
import html
conn = pymysql.connect(host='127.0.0.1',port=3306,user='root',password='eXYhzAWjyvy8grwM',db='game_process',charset='utf8mb4')
sql = "SELECT id,source,game_name, user_name, content, score FROM s_lcs_game_comments_qq WHERE source='qq_mobile'"
df = pd.read_sql(sql, conn)  # 返回的是 dataFrame 数据结构
conn.close()
df.head(10)

,id,source,game_name,user_name,content,score
0,1,qq_mobile,&nbsp;杀神免费版,游客,修复手机卡进行道具付费的漏洞！功能实用。速度很快。,5
1,2,qq_mobile,&nbsp;杀神免费版,游客,不错，不错这个游戏非常好玩随然登陆时间比较长但非常好玩的我看了一下登陆时间也差不多是一两分钟...,5
2,3,qq_mobile,&nbsp;杀神免费版,游客,新区，推荐。我叫一杯凉茶。一起玩,5
3,4,qq_mobile,&nbsp;杀神免费版,游客,请问通过人物，能不能找到账号和所在区。,5
4,5,qq_mobile,&nbsp;杀神免费版,游客,经常卡死。经常卡死。经常卡死。经常卡死。响应很慢。响应很慢。响应很慢。响应很慢。响应很慢。响...,1
5,6,qq_mobile,&nbsp;杀神免费版,游客,卡死,3
6,7,qq_mobile,&nbsp;杀神免费版,卿欲倾城,人好像有点少啊，多点人就好了，还不烧钱,4
7,8,qq_mobile,009-3D,游客,额...无语&hellip;&hellip;闪退。,1
8,9,qq_mobile,009-3D,( ✘_✘ )↯,,1
9,10,qq_mobile,100 Doors 2013,游客,为什么42关的密码知道是多少就是不开门呢？,5


In [85]:
df.to_csv('s_lcs_game_comments_qq.csv', encoding = 'utf-8', index = False)

In [83]:
df.dtypes # 查看 每列的数据类型

id            int64
source       object
game_name    object
user_name    object
content      object
score        object
dtype: object

In [29]:
df.index

RangeIndex(start=0, stop=337548, step=1)

In [30]:
df.columns

Index(['id', 'source', 'game_name', 'user_name', 'content', 'score'], dtype='object')

In [32]:
df.values

array([[1, 'qq_mobile', '&nbsp;杀神免费版', '游客', '修复手机卡进行道具付费的漏洞功能实用速度很快', '5'],
       [2, 'qq_mobile', '&nbsp;杀神免费版', '游客',
        '不错不错这个游戏非常好玩随然登陆时间比较长但非常好玩的我看了一下登陆时间也差不多是一两分钟左右吧欢迎大家来跟我一起玩', '5'],
       [3, 'qq_mobile', '&nbsp;杀神免费版', '游客', '新区推荐我叫一杯凉茶一起玩', '5'],
       ..., 
       [337546, 'qq_mobile', '쥬디의 오피스룩', '鈤岀', '', '5'],
       [337547, 'qq_mobile', '쥬디의 오피스룩', '£瘾’', '这个应用很不错', '4'],
       [337548, 'qq_mobile', '쥬디의 오피스룩', '喜性高冷i', '好玩', '5']], dtype=object)

In [36]:
df.describe()  # 计算 数值列 的各项数据

,id
count,337548.00000
mean,168774.50000
std,97441.85867
min,1.00000
25%,84387.75000
50%,168774.50000
75%,253161.25000
max,337548.00000


In [43]:
df[0:3]

,id,source,game_name,user_name,content,score
0,1,qq_mobile,&nbsp;杀神免费版,游客,修复手机卡进行道具付费的漏洞功能实用速度很快,5
1,2,qq_mobile,&nbsp;杀神免费版,游客,不错不错这个游戏非常好玩随然登陆时间比较长但非常好玩的我看了一下登陆时间也差不多是一两分钟左...,5
2,3,qq_mobile,&nbsp;杀神免费版,游客,新区推荐我叫一杯凉茶一起玩,5


In [20]:
def extract_text(x):
    soup = BeautifulSoup(x, 'html.parser')
    return soup.text

def clear_text(x):
    x = html.unescape(x)  # 反转义字符，显示真实内容。
    x = HanziConv.toSimplified(x) # 汉字繁体 转成 简体
    x = x.strip() # 去除字符 首尾的指定 字符
    return x

def remove_punctuation(x):  # 删除标点
    x = "".join([i for i in x if i not in punctuation]) # 遍历评论每个字符，若是标点，去掉，用空字符连接
    return re.sub(r'\s+', ' ', x) # 把 x 通过正则表达式 (\s 代表空格，+ 代表多个)，替换成 空字符

In [78]:
# 以 content 分组，选出每组的个数大于5的分组，并展示全部id 
df['content'] = df['content'].apply(clear_text)
df['content'] = df['content'].apply(remove_punctuation)

# cc = df[['content']].groupby(['content']).apply(len).reset_index()
# cc 
# # for x in df1:
# #     x    

df_content = df.groupby('content').size().reset_index()
df_content.columns = ['content', 'group_size']
df_content[df_content.group_size > 5]

,content,group_size
0,,407
135,11关怎么过,7
179,12关怎么过,7
226,13关怎么过,8
294,15关怎么过,7
373,18关怎么过,6
384,18年蕞火❤躺❤盈投❤资 wxLJ99NG 安✿全稳✿健可嘉葳❤要方鍅收溢达⑩倍❤以上,60
406,19关怎么过,6
421,1个G的手机运行起来毫无压力,17
637,24关怎么过,7
